# Notebook for Named Entity Recognition

Using spaCy for named entity recognition, we want to create relative frequency tables for the entities by year. At this point, we are only interested in the entities that appear most frequently.

Currently processes the "Fakespeak-ENG modified.xlsx" file (I've renamed my copy to "Fakespeak_ENG_modified.xlsx" to create a more consistent path), but will eventually be run on data from MisInfoText as well.

From the original data file, we use the following columns: ID, combinedLabel, originalTextType, originalBodyText, originalDateYear

We are processing text from the "originalBodyText" column.

In [ ]:
!pip install "spacy~=3.0.6"

In [ ]:
!python -m spacy download en_core_web_md

In [ ]:
!pip install spacy-entity-linker==1.0.3

In [ ]:
!python -m spacy_entity_linker "download_knowledge_base"

In [ ]:
import spacy
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Loading the articles

In [ ]:
input = '/content/drive/My Drive/fake_news_over_time/Fakespeak_ENG_modified.xlsx'

In [ ]:
fakespeak_df = pd.read_excel(input, sheet_name="Working", usecols=['ID', 'combinedLabel', 'originalTextType', 'originalBodyText', 'originalDateYear'])

In [ ]:
fakespeak_df.head()

,ID,combinedLabel,originalTextType,originalBodyText,originalDateYear
0,Politifact_FALSE_Social media_687276,False,Social media,Mexico is paying for the Wall through the new ...,2019
1,Politifact_FALSE_Social media_25111,False,Social media,"Chuck Schumer: ""why should American citizens b...",2019
2,Politifact_FALSE_Social media_735424,False,Social media,Billions of dollars are sent to the State of C...,2019
3,Politifact_FALSE_Social media_594307,False,Social media,If 50 Billion $$ were set aside to go towards ...,2019
4,Politifact_FALSE_Social media_839325,False,Social media,Huge@#CD 9 news. \n@ncsbe\n sent letter to eve...,2019


## Tagging named entities using spaCy

To make up for the difficulties of consolidating similar named entities, we use spaCy's large web model to ensure higher tagging accuracy in the initial NER step.

Documentation for entityLinker: https://github.com/egerber/spaCy-entity-linker

In [ ]:
# load spacy model
nlp = spacy.load("en_core_web_md")

# add custom entityLinker pipeline
nlp.add_pipe("entityLinker", last=True)

In [ ]:
# lists to hold spacy output elements
entities = []
ids = []
urls = []
years = []

for index, row in fakespeak_df.iterrows():
  article = nlp(row['originalBodyText'])
  year = row['originalDateYear']

  for ent in article._.linkedEntities:
    entities.append(ent.get_label())
    ids.append(ent.get_id())
    urls.append(ent.get_url())
    years.append(year)

In [ ]:
# dictionary to map the lists
tags = {
    'Entity': entities,
    'Wikidata_id': ids,
    'Wikidata_url': urls,
    'Year': years
}

# create a new dataframe containing the named entities
ner_df = pd.DataFrame(tags)

In [ ]:
ner_df.head()

,Entity,Wikidata_id,Wikidata_url,Year
0,Mexico,96,https://www.wikidata.org/wiki/Q96,2019
1,The Wall,27964590,https://www.wikidata.org/wiki/Q27964590,2019
2,United States–Mexico–Canada Agreement,56839716,https://www.wikidata.org/wiki/Q56839716,2019
3,The Wall,27964590,https://www.wikidata.org/wiki/Q27964590,2019
4,parking lot,6501349,https://www.wikidata.org/wiki/Q6501349,2019


## Filter dataframes by year and named entities
Currently, entityLinker catches all entities, not just proper nouns. To get around this, we first create dataframes filtering by year, then get the POS tags using spacy. This will then allow us to filter the dataframes further by excluding any counted nouns.

In [100]:
# create filtered dataframes
ner_19_df = ner_df[ner_df['Year'] == 2019]
ner_20_df = ner_df[ner_df['Year'] == 2020]
ner_21_df = ner_df[ner_df['Year'] == 2021]
ner_22_df = ner_df[ner_df['Year'] == 2022]
ner_23_df = ner_df[ner_df['Year'] == 2023]
ner_24_df = ner_df[ner_df['Year'] == 2024]

In [ ]:
# helper function for counting entities in each year
def get_count(df):
  df['Count'] = df.groupby(['Entity'])['Wikidata_id'].transform('count')
  sorted_df = df.sort_values(by=['Count', 'Entity', 'Wikidata_id'], ascending=False)
  unique_df = sorted_df.drop_duplicates()

  return unique_df

In [ ]:
# from each dataframe, obtain the counts of entities, sort by count, then keep unique values
# dropping N/A values to account for error in entityLinker tagging
ents_19_df = get_count(ner_19_df).dropna()
ents_20_df = get_count(ner_20_df).dropna()
ents_21_df = get_count(ner_21_df).dropna()
ents_22_df = get_count(ner_22_df).dropna()
ents_23_df = get_count(ner_23_df).dropna()
ents_24_df = get_count(ner_24_df).dropna()

In [103]:
ents_19_df.head()

,Entity,Wikidata_id,Wikidata_url,Year,Count
180,Donald Trump,22686,https://www.wikidata.org/wiki/Q22686,2019,94.0
127,United States of America,30,https://www.wikidata.org/wiki/Q30,2019,87.0
43,human,5,https://www.wikidata.org/wiki/Q5,2019,78.0
66,year,577,https://www.wikidata.org/wiki/Q577,2019,68.0
59,Democratic Party,29552,https://www.wikidata.org/wiki/Q29552,2019,55.0


In [ ]:
tagger = spacy.load("en_core_web_md")

In [104]:
ents_19_df['POS'] = [doc[0].pos_ for doc in tagger.pipe(ents_19_df['Entity'].tolist())]
ents_20_df['POS'] = [doc[0].pos_ for doc in tagger.pipe(ents_20_df['Entity'].tolist())]
ents_21_df['POS'] = [doc[0].pos_ for doc in tagger.pipe(ents_21_df['Entity'].tolist())]
ents_22_df['POS'] = [doc[0].pos_ for doc in tagger.pipe(ents_22_df['Entity'].tolist())]
ents_23_df['POS'] = [doc[0].pos_ for doc in tagger.pipe(ents_23_df['Entity'].tolist())]
ents_24_df['POS'] = [doc[0].pos_ for doc in tagger.pipe(ents_24_df['Entity'].tolist())]

In [106]:
# filter dataframes by proper noun only
ents_19_df = ents_19_df[ents_19_df['POS'] == 'PROPN']
ents_20_df = ents_20_df[ents_20_df['POS'] == 'PROPN']
ents_21_df = ents_21_df[ents_21_df['POS'] == 'PROPN']
ents_22_df = ents_22_df[ents_22_df['POS'] == 'PROPN']
ents_23_df = ents_23_df[ents_23_df['POS'] == 'PROPN']
ents_24_df = ents_24_df[ents_24_df['POS'] == 'PROPN']

In [107]:
ents_19_df.head()

,Entity,Wikidata_id,Wikidata_url,Year,Count,POS
180,Donald Trump,22686,https://www.wikidata.org/wiki/Q22686,2019,94.0,PROPN
127,United States of America,30,https://www.wikidata.org/wiki/Q30,2019,87.0,PROPN
59,Democratic Party,29552,https://www.wikidata.org/wiki/Q29552,2019,55.0,PROPN
428,United States Congress,11268,https://www.wikidata.org/wiki/Q11268,2019,38.0,PROPN
192,Andrew John Henry Way,17641254,https://www.wikidata.org/wiki/Q17641254,2019,25.0,PROPN


In [108]:
# helper function to calculate frequency in percentage
def get_prop(df):
  df['Proportion'] = df['Count'] / df['Count'].sum()

  return df

In [ ]:
ents_19_df = get_prop(ents_19_df)
ents_20_df = get_prop(ents_20_df)
ents_21_df = get_prop(ents_21_df)
ents_22_df = get_prop(ents_22_df)
ents_23_df = get_prop(ents_23_df)
ents_24_df = get_prop(ents_24_df)

In [110]:
ents_19_df.head()

,Entity,Wikidata_id,Wikidata_url,Year,Count,POS,Proportion
180,Donald Trump,22686,https://www.wikidata.org/wiki/Q22686,2019,94.0,PROPN,0.056153
127,United States of America,30,https://www.wikidata.org/wiki/Q30,2019,87.0,PROPN,0.051971
59,Democratic Party,29552,https://www.wikidata.org/wiki/Q29552,2019,55.0,PROPN,0.032855
428,United States Congress,11268,https://www.wikidata.org/wiki/Q11268,2019,38.0,PROPN,0.022700
192,Andrew John Henry Way,17641254,https://www.wikidata.org/wiki/Q17641254,2019,25.0,PROPN,0.014934


## Write results to Excel spreadsheet

In [ ]:
!pip install xlsxwriter

In [112]:
output = '/content/drive/My Drive/fake_news_over_time/named_entities_frequency.xlsx'

In [114]:
# create excel writer object to initialize new workbook
writer = pd.ExcelWriter(output, engine="xlsxwriter")

# write dataframes to different worksheets
ents_19_df.to_excel(writer, sheet_name="2019", columns=['Entity', 'Wikidata_id', 'Wikidata_url', 'Count', 'Proportion'], index=False)
ents_20_df.to_excel(writer, sheet_name="2020", columns=['Entity', 'Wikidata_id', 'Wikidata_url', 'Count', 'Proportion'], index=False)
ents_21_df.to_excel(writer, sheet_name="2021", columns=['Entity', 'Wikidata_id', 'Wikidata_url', 'Count', 'Proportion'], index=False)
ents_22_df.to_excel(writer, sheet_name="2022", columns=['Entity', 'Wikidata_id', 'Wikidata_url', 'Count', 'Proportion'], index=False)
ents_23_df.to_excel(writer, sheet_name="2023", columns=['Entity', 'Wikidata_id', 'Wikidata_url', 'Count', 'Proportion'], index=False)
ents_24_df.to_excel(writer, sheet_name="2024", columns=['Entity', 'Wikidata_id', 'Wikidata_url', 'Count', 'Proportion'], index=False)

# close the excel writer and output file
writer.close()